In [2]:
import pandas as pd
import duckdb

ModuleNotFoundError: No module named 'duckdb'

176. Вторая по величине зарплата  
Напишите решение для поиска второй по величине уникальной заработной платы в Employeeтаблице. Если второй по величине заработной платы нет, верните значение  null (return None in Pandas).

In [2]:
data = [[1, 100], [2, 200], [3, 300]]
employee = pd.DataFrame(data, columns=['id', 'salary']).astype({'id':'int64', 'salary':'int64'})
employee.head()

,id,salary
0,1,100
1,2,200
2,3,300


In [17]:
data = [[1, 0], [2, 0], [3, 1]]
employee = pd.DataFrame(data, columns=['id', 'salary']).astype({'id':'int64', 'salary':'int64'})
employee.head()

,id,salary
0,1,0
1,2,0
2,3,1


In [ ]:
# мое решение
employee = employee.drop_duplicates(subset=['salary']) 
employee['rank'] = employee['salary'].rank(method='dense', ascending=False).astype('int64')
data_1 = [1,2]
ranke_1 = pd.DataFrame(data_1, columns=['ranke_1']).astype({'ranke_1':'int64'})
union_table = ranke_1.merge(employee, left_on='ranke_1', right_on='rank', how='left')
union_table[union_table['ranke_1'] == 2][['salary']].rename(columns={'salary': 'SecondHighestSalary'})

In [ ]:
# решение GPT
salaries = employee['salary'].drop_duplicates().sort_values(ascending=False)
result = (
    salaries.iloc[1:2]
    .to_frame(name='SecondHighestSalary')
)
if result.empty:
    result = pd.DataFrame({'SecondHighestSalary': [None]})

,SecondHighestSalary
0,0


In [19]:
query = """
with t1 as (
    select 
        salary,
        dense_rank() over(order by salary desc) as rank
    from employee
    )
select
    case when count(*) < 1 then null
         else max(salary) end as SecondHighestSalary
from t1
where rank > 1
"""
result = duckdb.query(query).to_df()
result

,SecondHighestSalary
0,0


177. N-я самая высокая зарплата  
Напишите решение для поиска самой высокой уникальной заработной платы в таблице. Если уникальных зарплат  меньше, чем указано , верните значение .nthEmployeennull

In [8]:
N = 2
col_name = f'getNthHighestSalary({N})'
salaries = employee['salary'].drop_duplicates().sort_values(ascending=False)
if N > 0 and len(salaries) >= N:
    result = salaries.iloc[N-1:N].to_frame(name=col_name)
else:
    result = pd.DataFrame({col_name: [None]})
result

,getNthHighestSalary(2)
1,200


In [ ]:
N = 2
query = """
with t1 as (
    select 
        e.salary,
        dense_rank() over(order by e.salary desc) as rank
    from employee e
    )
select
    case when count(*) < 1 or N <= 0 then null
        else max(t1.salary) end as SecondHighestSalary
from t1
where rank > N - 1
"""
result = duckdb.query(query).to_df()
result

178. Ранговые баллы  
Напишите решение для определения ранга оценок. Ранжирование должно рассчитываться в соответствии со следующими правилами:  
- Оценки следует располагать в порядке убывания.  
- Если между двумя результатами есть ничья, оба показателя должны иметь одинаковый рейтинг.  
- В случае ничьей следующий номер в рейтинге должен быть следующим целым числом. Другими словами, между рангами не должно быть разрывов.  
Возвращает таблицу результатов, отсортированную в scoreпорядке убывания.

In [4]:
data = [[1, 3.5], [2, 3.65], [3, 4.0], [4, 3.85], [5, 4.0], [6, 3.65]]
scores = pd.DataFrame(data, columns=['id', 'score']).astype({'id':'Int64', 'score':'Float64'})
scores

,id,score
0,1,3.5
1,2,3.65
2,3,4.0
3,4,3.85
4,5,4.0
5,6,3.65


In [7]:
query = """
select
    score,
    dense_rank() over (order by score desc) as rank
from 
scores
"""
result = duckdb.query(query).to_df()
result

,score,rank
0,4.00,1
1,4.00,1
2,3.85,2
3,3.65,3
4,3.65,3
5,3.50,4


In [11]:
scores['rank'] = scores['score'].rank(method='dense', ascending=False).astype('Int64')
scores.sort_values('rank')[['score', 'rank']]

,score,rank
2,4.0,1
4,4.0,1
3,3.85,2
1,3.65,3
5,3.65,3
0,3.5,4


180. Последовательные номера  
Найдите все числа, которые встречаются как минимум три раза подряд.

In [13]:
data = [[1, 1], [2, 1], [3, 1], [4, 2], [5, 1], [6, 2], [7, 2]]
logs = pd.DataFrame(data, columns=['id', 'num']).astype({'id':'Int64', 'num':'Int64'})
logs

,id,num
0,1,1
1,2,1
2,3,1
3,4,2
4,5,1
5,6,2
6,7,2


In [ ]:
# мое решение
numbers = logs['num']
if len(numbers) > 0:
    result = []
    a = numbers[0]
    b = 1
    for i in range(1, len(numbers)):
        c = numbers[i]
        if c == a:
            b = b + 1
        else:
            b = 1
        a = c
        print(b)
        if b >= 3:
            result.append(numbers[i])
            b = 1
else:
    result = []
result_df = pd.DataFrame(result, columns=['ConsecutiveNums'])
result_df[['ConsecutiveNums']].drop_duplicates()

In [ ]:
# улучшение моего решения через цикл
nums = logs['num'].tolist()
result = []
count = 1
for i in range(1, len(nums)):
    if nums[i] == nums[i-1]:
        count += 1
        if count == 3:
            result.append(nums[i])
    else:
        count = 1
pd.DataFrame({'ConsecutiveNums': result})

In [ ]:
# еще одно решение GPT, через pandas
result = logs.loc[
    (logs['num'] == logs['num'].shift(1)) &
    (logs['num'] == logs['num'].shift(-1)),
    'num'
].drop_duplicates()
pd.DataFrame({'ConsecutiveNums': result})

In [ ]:
query = """
with t1 as(
    select 
        num,
        lag(num, 1, 0) over (order by id) as prev_1,
        lag(num, 2, 0) over (order by id) as prev_2
    from logs)
select distinct num as ConsecutiveNums
from t1
where num = prev_1 and num = prev_2
"""
result = duckdb.query(query).to_df()
result

184. Напишите решение для поиска сотрудников с самой высокой заработной платой в каждом из отделов.

In [4]:
data = [[1, 'Joe', 70000, 1], [2, 'Jim', 90000, 1], [3, 'Henry', 80000, 2], [4, 'Sam', 60000, 2], [5, 'Max', 90000, 1]]
employee = pd.DataFrame(data, columns=['id', 'name', 'salary', 'departmentId']).astype({'id':'Int64', 'name':'object', 'salary':'Int64', 'departmentId':'Int64'})
data = [[1, 'IT'], [2, 'Sales']]
department = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'Int64', 'name':'object'})

In [5]:
employee

,id,name,salary,departmentId
0,1,Joe,70000,1
1,2,Jim,90000,1
2,3,Henry,80000,2
3,4,Sam,60000,2
4,5,Max,90000,1


In [6]:
department

,id,name
0,1,IT
1,2,Sales


In [13]:
# мое решение 
employee['rank'] = employee.groupby('departmentId')['salary'].rank(ascending=False).astype('int64')
merged = employee.merge(department, how='left', left_on='departmentId', right_on='id')
merged[merged['rank'] == 1].rename(columns={
    'name_y': 'Department', 
    'name_x': 'Employee', 
    'salary': 'Salary'
    })[['Department', 'Employee', 'Salary']]

,Department,Employee,Salary
1,IT,Jim,90000
2,Sales,Henry,80000
4,IT,Max,90000


In [8]:
# решение с сайта
merged = employee.merge(department, left_on='departmentId', right_on='id', how='left')
highest_salary = merged.loc[merged.groupby('departmentId')['salary'].transform('max') == merged['salary']]
result = highest_salary[['name_x', 'salary', 'name_y']].rename(columns={ 
    'name_y': 'Department',  # department name column
    'name_x': 'Employee',    # employee name column
    'salary': 'Salary'       # salary column
})
result[['Department', 'Employee', 'Salary']]

,Department,Employee,Salary
1,IT,Jim,90000
2,Sales,Henry,80000
4,IT,Max,90000


In [10]:
query = """
with t1 as(
    select
        d.name as Department,
        e.name as Employee,
        e.salary as Salary,
        dense_rank() over(partition by departmentId order by salary desc) as salary_rank
    from employee e
    join department d
    on e.departmentId = d.id
)
select
    Department,
    Employee,
    Salary
from t1
where salary_rank = 1
"""
result = duckdb.query(query).to_df()
result

,Department,Employee,Salary
0,Sales,Henry,80000
1,IT,Jim,90000
2,IT,Max,90000


550. Напишите решение, которое выведет долю игроков, повторно вошедших в систему на следующий день после первого входа, округлив до двух знаков после запятой . Другими словами, вам нужно определить количество игроков, вошедших в систему на следующий день после первого входа, и разделить его на общее количество игроков.

In [2]:
data = [[1, 2, '2016-03-01', 5], [1, 2, '2016-03-02', 6], [2, 3, '2017-06-25', 1], [3, 1, '2016-03-02', 0], [3, 4, '2018-07-03', 5]]
activity = pd.DataFrame(data, columns=['player_id', 'device_id', 'event_date', 'games_played']).astype({'player_id':'Int64', 'device_id':'Int64', 'event_date':'datetime64[ns]', 'games_played':'Int64'})
activity.head()

,player_id,device_id,event_date,games_played
0,1,2,2016-03-01,5
1,1,2,2016-03-02,6
2,2,3,2017-06-25,1
3,3,1,2016-03-02,0
4,3,4,2018-07-03,5


In [ ]:
query = """
with t as(
    select
        player_id,
        min(event_date) as min_date
    from activity
    group by player_id)
select
    round(count(distinct t1.player_id) * 1.0 / (select count(distinct player_id) from activity),2) as fraction
from t t1 join activity t2
    on t1.min_date + 1 = t2.event_date and t1.player_id = t2.player_id
"""
result = duckdb.query(query).to_df()
result

In [41]:
activity['min_date'] = activity.groupby('player_id', as_index=False).event_date.transform(min)
pd.DataFrame([round(activity[activity['event_date'] == activity['min_date'] + pd.Timedelta(days=1)].player_id.nunique() / activity.player_id.nunique(),2)], columns=['fraction'])

C:\Users\user\AppData\Local\Temp\ipykernel_1916\3943309026.py:1: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  activity['min_date'] = activity.groupby('player_id', as_index=False).event_date.transform(min)


,fraction
0,0.33


570. Менеджеры, имеющие не менее 5 непосредственных подчиненных. Напишите решение для поиска менеджеров, у которых в подчинении не менее пяти человек .

In [16]:
data = [[101, 'John', 'A', None], [102, 'Dan', 'A', 101], [103, 'James', 'A', 101], [104, 'Amy', 'A', 101], [105, 'Anne', 'A', 101], [106, 'Ron', 'B', 101]]
employee = pd.DataFrame(data, columns=['id', 'name', 'department', 'managerId']).astype({'id':'Int64', 'name':'object', 'department':'object', 'managerId':'Int64'})
employee

,id,name,department,managerId
0,101,John,A,<NA>
1,102,Dan,A,101
2,103,James,A,101
3,104,Amy,A,101
4,105,Anne,A,101
5,106,Ron,B,101


In [4]:
query = """
with t as(
    select
        managerId
    from employee
    group by managerId
    having count(managerId) >= 5)
select name
from t join employee e
    on t.managerId = e.id
"""
result = duckdb.query(query).to_df()
result

,name
0,John


In [15]:
employee['count_emp'] = employee.groupby('managerId', as_index=False).managerId.transform('count')
t1 = employee[employee['count_emp'] >= 5][['managerId']].drop_duplicates()
t1.merge(employee, how='inner', left_on='managerId', right_on='id')[['name']]

,name
0,John


In [17]:
managers = employee.groupby('managerId', as_index=False).agg(reporting=('id', 'count'),).query('5 <= reporting')['managerId']
employee[employee['id'].isin(managers)][['name']]

,name
0,John


585. Инвестиции в 2016 году. Напишите решение, которое выведет сумму всех общих инвестиционных ценностей за 2016 год tiv_2016для всех страхователей, которые:  
- имеют такое же tiv_2015значение, как у одного или нескольких других страхователей  
- не находятся в том же городе, что и другие страхователи (т. е. пары атрибутов (lat, lon) должны быть уникальными).

In [56]:
data = [[1, 10, 5, 10, 10], [2, 20, 20, 20, 20], [3, 10, 30, 20, 20], [4, 10, 40, 40, 40]]
insurance = pd.DataFrame(data, columns=['pid', 'tiv_2015', 'tiv_2016', 'lat', 'lon']).astype({'pid':'Int64', 'tiv_2015':'Float64', 'tiv_2016':'Float64', 'lat':'Float64', 'lon':'Float64'})
insurance

,pid,tiv_2015,tiv_2016,lat,lon
0,1,10.0,5.0,10.0,10.0
1,2,20.0,20.0,20.0,20.0
2,3,10.0,30.0,20.0,20.0
3,4,10.0,40.0,40.0,40.0


In [ ]:
# мое решение
insurance['duble_lat_lon'] = insurance.groupby(['lat', 'lon'], as_index=False).pid.transform('count')
insurance['duble_tiv_2015'] = insurance.groupby('tiv_2015', as_index=False).pid.transform('count')
insurance[(insurance['duble_lat_lon'] == 1) & (insurance['duble_tiv_2015'] > 1)][['tiv_2016']].sum().round(2).to_frame(name='tiv_2016')

,tiv_2016
tiv_2016,45.0


In [ ]:
# решение с сайта, с моей точки зрения - не очень очевидное
df = pd.merge(insurance[insurance.duplicated('tiv_2015', keep = False)], insurance.drop_duplicates(['lat','lon'], keep = False), on = 'pid').iloc[:, [0, 2]]
pd.DataFrame({'tiv_2016': [round(df['tiv_2016_x'].sum(), 2)]})

,tiv_2016
0,45.0


In [4]:
query = """
WITH CheckedRows AS (
    SELECT
        tiv_2016,
        COUNT(*) OVER (PARTITION BY tiv_2015) AS tiv_2015_count,
        COUNT(*) OVER (PARTITION BY lat, lon) AS loc_count
    FROM
        insurance
)
SELECT
    ROUND(SUM(tiv_2016)::numeric, 2) AS tiv_2016
FROM
    CheckedRows
WHERE
    tiv_2015_count > 1
    AND loc_count = 1
"""
duckdb.query(query).to_df()

,tiv_2016
0,45.0


In [7]:
query = """
select
round(SUM(tiv_2016)::numeric,2) AS tiv_2016
from insurance
where tiv_2015 in (select tiv_2015 from insurance group by tiv_2015 having count(tiv_2015)>1)
and (lat, lon) in (select lat, lon from insurance group by lat, lon having count(*)=1)
"""
duckdb.query(query).to_df()

,tiv_2016
0,45.0


602. Запросы на добавление в друзья II: у кого больше друзей. Напишите решение, которое поможет найти людей с наибольшим количеством друзей и само количество друзей. Тестовые примеры составлены таким образом, что только у одного человека может быть наибольшее количество друзей.

In [15]:
data = [[1, 2, '2016/06/03'], [1, 3, '2016/06/08'], [2, 3, '2016/06/08'], [3, 4, '2016/06/09']]
request_accepted = pd.DataFrame(data, columns=['requester_id', 'accepter_id', 'accept_date']).astype({'requester_id':'Int64', 'accepter_id':'Int64', 'accept_date':'datetime64[ns]'})
request_accepted

,requester_id,accepter_id,accept_date
0,1,2,2016-06-03
1,1,3,2016-06-08
2,2,3,2016-06-08
3,3,4,2016-06-09


In [17]:
request_accepted.merge(request_accepted, how='left', left_on='requester_id', right_on='accepter_id')

,requester_id_x,accepter_id_x,accept_date_x,requester_id_y,accepter_id_y,accept_date_y
0,1,2,2016-06-03,<NA>,<NA>,NaT
1,1,3,2016-06-08,<NA>,<NA>,NaT
2,2,3,2016-06-08,1,2,2016-06-03
3,3,4,2016-06-09,1,3,2016-06-08
4,3,4,2016-06-09,2,3,2016-06-08


In [ ]:
# если есть только один человек с максимальным количеством друзей
query = """
with t1 as
(
    SELECT requester_id as id FROM request_accepted
    UNION ALL
    SELECT accepter_id as id FROM request_accepted
)    
select 
    id,
    count(id) as num
from t1
group by id
order by num desc
limit 1
"""
duckdb.query(query).to_df()

,id,num
0,3,3


In [16]:
# если у нескольких людей есть максимальное количество друзей
query = """
with t1 as
(
    SELECT requester_id as id FROM request_accepted
    UNION ALL
    SELECT accepter_id as id FROM request_accepted
),   
t2 as
(
    select 
        id,
        count(id) over(partition by id) as count_friends
    from t1
)
select distinct 
    id, 
    count_friends as num
from t2
where count_friends = (select max(count_friends) from t2)
"""
duckdb.query(query).to_df()

,id,num
0,3,3


608. Узел дерева. Каждый узел дерева может быть одного из трёх типов:  
"Листовой": если узел является листовым.  
"Корневой": если узел является корневым.  
"Внутренний": Если узел не является ни листовым, ни корневым.  
Напишите решение, которое будет сообщать тип каждого узла в дереве.

In [2]:
data = [[1, None], [2, 1], [3, 1], [4, 2], [5, 2]]
tree = pd.DataFrame(data, columns=['id', 'p_id']).astype({'id':'Int64', 'p_id':'Int64'})
tree

,id,p_id
0,1,<NA>
1,2,1
2,3,1
3,4,2
4,5,2


In [3]:
def classify_node(row):
    if pd.isna(row['p_id']):
        return 'Root'
    elif row['id'] not in tree['p_id'].values:
        return 'Leaf'
    else:
        return 'Inner'
tree['type'] = tree.apply(classify_node, axis=1)
tree[['id', 'type']]

,id,type
0,1,Root
1,2,Inner
2,3,Leaf
3,4,Leaf
4,5,Leaf


In [4]:
query = """
with t1 as(
    select
        p_id as id,
        count(id) as count_compound
    from tree
    group by p_id
)
select
    id,
    case 
    when p_id is null then 'Root'
    when id in (select id from t1) then 'Inner'
    else 'Leaf' 
    end
    as type
from tree
"""
duckdb.query(query).to_df()

,id,type
0,1,Root
1,2,Inner
2,3,Leaf
3,4,Leaf
4,5,Leaf


626. Поменяйтесь местами. Напишите программу, которая меняет местами идентификаторы каждых двух последовательных студентов. Если количество студентов нечетное, идентификатор последнего студента не меняется местами с остальными.

In [5]:
data = [[1, 'Abbot'], [2, 'Doris'], [3, 'Emerson'], [4, 'Green'], [5, 'Jeames']]
seat = pd.DataFrame(data, columns=['id', 'student']).astype({'id':'Int64', 'student':'object'})
seat

,id,student
0,1,Abbot
1,2,Doris
2,3,Emerson
3,4,Green
4,5,Jeames


In [6]:
def exchange(x):
    if x % 2 == 0: return x - 1
    elif x == len(seat): return x
    else: return x + 1
seat['id'] = seat['id'].apply(exchange)
seat.sort_values(by='id')

,id,student
1,1,Doris
0,2,Abbot
3,3,Green
2,4,Emerson
4,5,Jeames


In [7]:
query = """
with t1 as(
    select max(id) as id
    from seat
)
select
    case when id % 2 = 0 then id - 1
    when id = (select id from t1) then id
    else id + 1
    end as id,
    student
from seat
order by id
"""
duckdb.query(query).to_df()

,id,student
0,1,Abbot
1,2,Doris
2,3,Emerson
3,4,Green
4,5,Jeames


1045. Покупатели, которые приобрели все товары. пишите решение для получения идентификаторов клиентов из Customer таблицы, которые купили все продукты в Product таблице.

In [8]:
data = [[1, 5], [2, 6], [3, 5], [3, 6], [1, 6]]
customer = pd.DataFrame(data, columns=['customer_id', 'product_key']).astype({'customer_id':'Int64', 'product_key':'Int64'})
data = [[5], [6]]
product = pd.DataFrame(data, columns=['product_key']).astype({'product_key':'Int64'})
customer

,customer_id,product_key
0,1,5
1,2,6
2,3,5
3,3,6
4,1,6


In [9]:
data

[[5], [6]]

In [10]:
query = """
with t1 as(
    select count(distinct(product_key)) from product
)
select customer_id 
from customer
group by(customer_id)
having count(distinct(product_key)) >= (select * from t1)
"""
duckdb.query(query).to_df()

,customer_id
0,3
1,1


In [26]:
count_prod = len(product)
customer.groupby('customer_id', as_index=False).agg({'product_key': 'nunique'}).query('product_key >= @count_prod')[['customer_id']]

,customer_id
0,1
2,3


1070. Анализ продаж продукции III. Напишите решение, которое позволит найти все продажи, совершённые в первый год по каждому товару. Для каждого product_id определите самый ранний year в таблице Sales. Возврат всех записей о продажах этого товара за этот год.

In [27]:
data = [[1, 100, 2008, 10, 5000], [2, 100, 2009, 12, 5000], [7, 200, 2011, 15, 9000]]
sales = pd.DataFrame(data, columns=['sale_id', 'product_id', 'year', 'quantity', 'price']).astype({'sale_id':'Int64', 'product_id':'Int64', 'year':'Int64', 'quantity':'Int64', 'price':'Int64'})
sales

,sale_id,product_id,year,quantity,price
0,1,100,2008,10,5000
1,2,100,2009,12,5000
2,7,200,2011,15,9000


In [28]:
query = """
with t1 as(    
    select
        product_id,
        year as  first_year,
        quantity,
        price,
        dense_rank() over(partition by product_id order by year) as  rank_year
    from sales
)
select
    product_id,
    first_year,
    quantity,
    price
from t1
where rank_year = 1
"""
duckdb.query(query).to_df()

,product_id,first_year,quantity,price
0,200,2011,15,9000
1,100,2008,10,5000


In [33]:
sales['rank_year'] = sales.groupby('product_id')['year'].transform(lambda x: x.rank(method='dense'))
sales[sales['rank_year'] == 1].rename(columns={'year': 'first_year'})[['product_id', 'first_year', 'quantity', 'price']]

,product_id,first_year,quantity,price
0,100,2008,10,5000
2,200,2011,15,9000


1158. Анализ рынка I. Напишите решение для поиска для каждого пользователя, дату присоединения и количество заказов, сделанных им в качестве покупателя в 2019.

In [34]:
data = [[1, '2018-01-01', 'Lenovo'], [2, '2018-02-09', 'Samsung'], [3, '2018-01-19', 'LG'], [4, '2018-05-21', 'HP']]
users = pd.DataFrame(data, columns=['user_id', 'join_date', 'favorite_brand']).astype({'user_id':'Int64', 'join_date':'datetime64[ns]', 'favorite_brand':'object'})
data = [[1, '2019-08-01', 4, 1, 2], [2, '2018-08-02', 2, 1, 3], [3, '2019-08-03', 3, 2, 3], [4, '2018-08-04', 1, 4, 2], [5, '2018-08-04', 1, 3, 4], [6, '2019-08-05', 2, 2, 4]]
orders = pd.DataFrame(data, columns=['order_id', 'order_date', 'item_id', 'buyer_id', 'seller_id']).astype({'order_id':'Int64', 'order_date':'datetime64[ns]', 'item_id':'Int64', 'buyer_id':'Int64', 'seller_id':'Int64'})
data = [[1, 'Samsung'], [2, 'Lenovo'], [3, 'LG'], [4, 'HP']]
items = pd.DataFrame(data, columns=['item_id', 'item_brand']).astype({'item_id':'Int64', 'item_brand':'object'})
users

,user_id,join_date,favorite_brand
0,1,2018-01-01,Lenovo
1,2,2018-02-09,Samsung
2,3,2018-01-19,LG
3,4,2018-05-21,HP


In [35]:
orders

,order_id,order_date,item_id,buyer_id,seller_id
0,1,2019-08-01,4,1,2
1,2,2018-08-02,2,1,3
2,3,2019-08-03,3,2,3
3,4,2018-08-04,1,4,2
4,5,2018-08-04,1,3,4
5,6,2019-08-05,2,2,4


In [36]:
items

,item_id,item_brand
0,1,Samsung
1,2,Lenovo
2,3,LG
3,4,HP


In [ ]:
query = """
select
    user_id as buyer_id,
    join_date,
    sum(case when to_char(order_date, 'YYYY') = '2019' then 1 else 0 end) as orders_in_2019
from orders o
full outer join users u
on u.user_id = o.buyer_id
group by user_id, join_date
order by buyer_id
"""
duckdb.query(query).to_df()

In [48]:
orders_2019 = orders[orders['order_date'].dt.year == 2019].groupby('buyer_id').agg(orders_in_2019=('order_date', 'count'))
users.rename(columns={'user_id': 'buyer_id'}).merge(orders_2019, how='outer', on='buyer_id').fillna(0)[['buyer_id', 'join_date', 'orders_in_2019']]

,buyer_id,join_date,orders_in_2019
0,1,2018-01-01,1.0
1,2,2018-02-09,2.0
2,3,2018-01-19,0.0
3,4,2018-05-21,0.0


1164. Цена товара на указанную дату. Изначально все продукты имеют цену 10. Напишите решение для нахождения цен на все продукты на дату 2019-08-16.

In [3]:
data = [[1, 20, '2019-08-14'], [2, 50, '2019-08-14'], [1, 30, '2019-08-15'], [1, 35, '2019-08-16'], [2, 65, '2019-08-17'], [3, 20, '2019-08-18']]
products = pd.DataFrame(data, columns=['product_id', 'new_price', 'change_date']).astype({'product_id':'Int64', 'new_price':'Int64', 'change_date':'datetime64[ns]'})
products

,product_id,new_price,change_date
0,1,20,2019-08-14
1,2,50,2019-08-14
2,1,30,2019-08-15
3,1,35,2019-08-16
4,2,65,2019-08-17
5,3,20,2019-08-18


In [4]:
query = """
with t1 as(
    select
        product_id,
        new_price as price,
        row_number() over(partition by product_id order by change_date desc) as row_price
    from products
    where change_date <= '2019-08-16'
),
t2 as(
    select distinct product_id from products
    )
select 
    t2.product_id,
    coalesce(t1.price, 10) as price
from t2 left join t1 on t1.product_id = t2.product_id and t1.row_price = 1
"""
duckdb.query(query).to_df()

,product_id,price
0,2,50
1,3,10
2,1,35


In [ ]:
df = products[products.change_date <= '2019-08-16']
latest_dates = df.groupby('product_id')['change_date'].transform('max')
df[df.change_date == latest_dates].merge(products.product_id.drop_duplicates(), how = 'right').fillna(10).rename(columns = {'new_price': 'price'}).iloc[:,[0,1]]

,product_id,price
0,1,35
1,2,50
2,3,10


1174. Немедленная доставка продуктов питания II. Если желаемая дата доставки клиента совпадает с датой заказа, то заказ считается немедленным; в противном случае он считается запланированным. Первым заказом клиента считается заказ с самой ранней датой оформления. Гарантируется, что у клиента будет ровно один первый заказ. Напишите решение для нахождения процента немедленных заказов в первых заказах всех клиентов, округленного до двух знаков после запятой .

In [24]:
data = [[1, 1, '2019-08-01', '2019-08-02'], [2, 2, '2019-08-02', '2019-08-02'], [3, 1, '2019-08-11', '2019-08-12'], [4, 3, '2019-08-24', '2019-08-24'], [5, 3, '2019-08-21', '2019-08-22'], [6, 2, '2019-08-11', '2019-08-13'], [7, 4, '2019-08-09', '2019-08-09']]
delivery = pd.DataFrame(data, columns=['delivery_id', 'customer_id', 'order_date', 'customer_pref_delivery_date']).astype({'delivery_id':'Int64', 'customer_id':'Int64', 'order_date':'datetime64[ns]', 'customer_pref_delivery_date':'datetime64[ns]'})
delivery

,delivery_id,customer_id,order_date,customer_pref_delivery_date
0,1,1,2019-08-01,2019-08-02
1,2,2,2019-08-02,2019-08-02
2,3,1,2019-08-11,2019-08-12
3,4,3,2019-08-24,2019-08-24
4,5,3,2019-08-21,2019-08-22
5,6,2,2019-08-11,2019-08-13
6,7,4,2019-08-09,2019-08-09


In [12]:
delivery.assign(firt_order = delivery.groupby('customer_id')['order_date'].transform(lambda x: x.rank(method='min'))).query('firt_order==1')


,delivery_id,customer_id,order_date,customer_pref_delivery_date,firt_order
0,1,1,2019-08-01,2019-08-02,1.0
1,2,2,2019-08-02,2019-08-02,1.0
4,5,3,2019-08-21,2019-08-22,1.0
6,7,4,2019-08-09,2019-08-09,1.0


In [ ]:
first_orders = delivery.loc[delivery.groupby('customer_id')['order_date'].idxmin()]
first_orders['immediate'] = first_orders['order_date'] == first_orders['customer_pref_delivery_date']
immediate_percent = first_orders['immediate'].mean() * 100

In [22]:
delivery.groupby('customer_id')['order_date'].idxmin()

customer_id
1    0
2    1
3    4
4    6
Name: order_date, dtype: int64

In [26]:
print(delivery.groupby('customer_id'))

In [23]:
first_orders['immediate'] = first_orders['order_date'] == first_orders['customer_pref_delivery_date']
immediate_percent = first_orders['immediate'].mean() * 100
immediate_percent

np.float64(50.0)

In [3]:
data = [[121, 'US', 'approved', 1000, '2018-12-18'], [122, 'US', 'declined', 2000, '2018-12-19'], [123, 'US', 'approved', 2000, '2019-01-01'], [124, 'DE', 'approved', 2000, '2019-01-07']]
transactions = pd.DataFrame(data, columns=['id', 'country', 'state', 'amount', 'trans_date']).astype({'id':'Int64', 'country':'object', 'state':'object', 'amount':'Int64', 'trans_date':'datetime64[ns]'})
transactions

,id,country,state,amount,trans_date
0,121,US,approved,1000,2018-12-18
1,122,US,declined,2000,2018-12-19
2,123,US,approved,2000,2019-01-01
3,124,DE,approved,2000,2019-01-07


In [4]:
df = transactions

In [7]:
amount[df.loc[x.index, 'state'] == 'approved']

NameError: name 'amount' is not defined